In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import os

import mytools as mt
import baikalfunctions as bf
import scheme_mar2023 as scheme

In [ ]:
#plt.style.use('Solarize_Light2')   # try another styles: 'classic'
plt.style.use('default')
plt.rcParams['figure.figsize'] = [15, 15]
#driver = 'C:/Users/redmi/'
driver = 'G:/'
pwd = os.path.join(driver, "1_Data1/sea/")
destData = os.path.join(pwd, "AllSeaGisOut.txt")
obrisData = os.path.join(driver, "1_Data1/obris/obris.dat")
batimetrFile = os.path.join(driver, "1_Data1/batimetria/b1.txt")
#os.listdir(pwd)

In [ ]:
obris = pd.read_csv(obrisData,
                    header=0,
                    na_values='--',
                    sep='\t',
                    decimal=',',
                    )
obris.head(3)

In [ ]:
df = pd.read_csv(os.path.join(pwd, "AllSeaO3.txt"),
                 skiprows=[1],
                 header=0,
                 # skip_blank_lines=True,
                 skipinitialspace=True,
                 na_values='',
                 sep='\t',
                 decimal='.',
                 # nrows=100000,
                 parse_dates=['DateTime'],
                 dayfirst=False,
                 )
df.info()

In [ ]:
df

In [ ]:
df['CO2flux'] = 0.00485 * df['dpCO2'] * np.power(df['winSpeed_rolling'], 2)
df['CH4flux'] = 0.0605 * df['dpCH4'] * np.power(df['winSpeed_rolling'], 2)

In [ ]:
param = 'dpCO2'

In [ ]:
plt.rcParams['figure.figsize'] = [10,5]
yr = 2016
data=df.loc[(df['DateTime'] > pd.to_datetime(f'03.06.{yr} 00:00', dayfirst=True)) & (df['DateTime'] < pd.to_datetime(f'03.06.{yr} 10:00', dayfirst=True)), :]
#plt.ylim(-450, -250)
plt.scatter('DateTime', param, s=1, data=data, c='blue')
plt.scatter('DateTime', 'pCO2Wtr', s=1, data=data, c='red')

In [ ]:
central_stations = {}
for key, coor in scheme.central.items():
    central_stations[key] = scheme.get_center(coor)
central_stations

In [ ]:
dfc = pd.DataFrame.from_dict(central_stations, orient='index')
dfc.columns = ['Latitude', 'Longitude']
dfp = pd.DataFrame.from_dict(scheme.points, orient='index')
dfp.columns = ['Latitude', 'Longitude']
dfp.reset_index(drop=False, inplace=True, names='place')

In [ ]:
KYA = scheme.points['KYA']
KHR = scheme.points['KHR']
south_middle_border_k = (KYA[0] - KHR[0]) / (KYA[1] - KHR[1])
south_middle_border_b = KYA[0] - south_middle_border_k * KYA[1]

In [ ]:
df['region'] = 'middle'
df.loc[(df['Longitude']*south_middle_border_k + south_middle_border_b) > df['Latitude'], 'region'] = 'south'
df.loc[df['Latitude'] > scheme.points['mACR'][0], 'region'] = 'north'

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
plt.scatter('Longitude', 'Latitude', data=df.loc[df['year'] == 2013,:], s=1, alpha=0.1)
plt.scatter('Longitude', 'Latitude', data=df.loc[df['region'] == 'south',:], c='r', s=1, alpha=0.03)
plt.scatter('Longitude', 'Latitude', data=df.loc[df['region'] == 'middle',:], c='g', s=1, alpha=0.03)
plt.scatter('Longitude', 'Latitude', data=df.loc[df['region'] == 'north',:], c='b', s=1, alpha=0.03)
#plt.scatter('Longitude', 'Latitude', data=dfj, s=2, c='green', alpha=0.1)
plt.scatter('Longitude', 'Latitude', data=dfc, s=20, c='red', alpha=1)
plt.scatter('Longitude', 'Latitude', data=dfp, s=10, c='darkred', alpha=1)
plt.plot('long', 'lat', 'b-', data=obris, alpha=1, linewidth=1)
plt.plot([KYA[1], KHR[1]], [KYA[0], KHR[0]], 'g-', data=obris, alpha=1, linewidth=1)
for name, coor in central_stations.items():
    plt.annotate(name, (coor[1]+0.002, coor[0]+0.004))
for name, coor in scheme.points.items():
    plt.annotate(name, (coor[1]+0.001, coor[0]+0.002))
plt.savefig('track_2_3.png')

In [ ]:
df['Station'] = ''
DLAT = 0.01
DLNG = 0.02

for k, v in central_stations.items():
    df.loc[(df['Latitude'].between(v[0]-DLAT, v[0]+DLAT) & df['Longitude'].between(v[1]-DLNG, v[1]+DLNG)), 'Station'] = k

In [ ]:
dfg = df.groupby(by=['region'])[param].agg(['count', 'mean', 'std', 'median'])
dfg

In [ ]:
dfg.to_csv(os.path.join(pwd, f'{param}RegionStationAllSpring.dat'), sep='\t')

In [ ]:
## vesx: RV x offset along longitude, vesy: RV y offset along latitude (meters)
df['dateSec'] = df['DateTime'].astype('int64')/10**9  ## seconds
df['dt'] = (df['dateSec'] - df['dateSec'].shift(1))

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
plt.scatter('DateTime', 'winSpeed_rolling', data=df, s=4, c='red')
plt.xlim(pd.to_datetime('2021-06-02 06:10'), pd.to_datetime('2021-06-03 01:30'))
plt.legend(loc='lower right')
plt.grid()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]
plt.scatter(df['Longitude'], df['Latitude'], s=10, c=df['depth'], marker='o', linewidth=0, alpha=0.5)
plt.plot(obris['long'], obris['lat'], 'b-')
plt.colorbar(alpha=1)

In [ ]:
longStep = 0.02
latStep = 0.01
df['long'] = (df['Longitude']/longStep).round()*longStep
df['lat'] = (df['Latitude']/latStep).round()*latStep

In [ ]:
df.columns

In [ ]:
#func_list = ['mean', 'count']
func_list = ['mean']
cols_to_calc = { #'pCO2Wtr': func_list,
                #'pCH4Wtr': func_list,
                #'cCO2Wtr': func_list,
                #'cCH4Wtr': func_list,
                'dpCO2': func_list,
                'dpCH4': func_list,
                'CO2flux': func_list,
                'CH4flux': func_list,
                #'pCO2Air': func_list,
                #'pCH4Air': func_list,
                'depth': ['mean'],
                'TempWtr': func_list,
                'winSpeed_rolling': func_list,
                param: func_list,}
res = df.groupby(['year', 'long', 'lat'], as_index=False).agg(cols_to_calc)
res.columns = res.columns.droplevel(1)
#res = df.groupby(['long', 'lat'], as_index=False).agg(cols_to_calc)
#res.columns = res.columns.droplevel(1)
res

In [ ]:
for yr in range(2013, 2025):
    t = res.loc[res['year'] == yr].shape
    print(f'{yr}: {t[0]}')

In [ ]:
res['square'] = bf.dist((0, res['lat']), (0.02, res['lat']))*1.11

In [ ]:
res['region'] = '2middle'
res.loc[(res['long']*south_middle_border_k + south_middle_border_b) > res['lat'], 'region'] = '1south'
res.loc[res['lat'] > scheme.points['mACR'][0], 'region'] = '3north'

In [ ]:
res['depthLevel'] = ''
res.loc[res['depth'].between(-100, 100), 'depthLevel'] = '0-100m'
res.loc[res['depth'].between(-200, -100), 'depthLevel'] = '100-200m'
res.loc[res['depth'].between(-400, -200), 'depthLevel'] = '200-400m'
res.loc[res['depth'].between(-2000, -400), 'depthLevel'] = '400+'
res

In [ ]:
param = 'CH4flux'
res.groupby(by=['depthLevel']).agg({param: ['count','mean', 'std', 'median']}).to_clipboard()

In [ ]:
res.groupby(by=['year', 'depthLevel']).agg({param: ['count', 'mean', 'std', 'median']}).to_clipboard()

In [ ]:
plt.rcParams['figure.figsize'] = [10,10]
#plt.plot('long', 'lat', data=obris, markersize=1, alpha=1)
#plt.scatter('long', 'lat', data=res, s=7, alpha=1, c=res[param], cmap='rainbow', marker='s', )
#for yr in [2013, 2016, 2017, 2018, 2021, 2022, 2023, 2024]:
for yr in [2022, 2023, 2024]:
    plt.plot('long', 'lat', data=obris, markersize=1, alpha=1)
    plt.scatter('long', 'lat', data=res.loc[res['year']==yr, :], s=20, alpha=0.8, c=res.loc[res['year']==yr, param], cmap='rainbow')
    plt.xlim(105, 106.4)
    plt.ylim(51.5, 52.5)
    plt.clim(0,10)
    plt.colorbar()
    plt.title(f'{param} {yr} malenkiy')
    plt.savefig(f'{param}Grid1_1km_malenkiy_{yr}.png')
    plt.close()

In [ ]:
plt.rcParams['figure.figsize'] = [10,10]
#plt.plot('long', 'lat', data=obris, markersize=1, alpha=1)
#plt.scatter('long', 'lat', data=res, s=7, alpha=1, c=res[param], cmap='rainbow', marker='s', )
#for yr in [2013, 2016, 2017, 2018, 2021, 2022, 2023, 2024]:
for yr in [2022, 2023, 2024]:
    plt.plot('long', 'lat', data=obris, markersize=1, alpha=1)
    plt.scatter('long', 'lat', data=res.loc[res['year']==yr, :], s=20, alpha=0.8, c=res.loc[res['year']==yr, param], cmap='rainbow')
    plt.xlim(106.8, 108)
    plt.ylim(52.5, 53.5)
    plt.clim(0,10)
    plt.colorbar()
    plt.title(f'{param} {yr} olkhon')
    plt.savefig(f'{param}Grid1_1km_olkhon_{yr}.png')
    plt.close()

In [ ]:
res

In [ ]:
func_list = ['mean']
cols_to_calc = { #'pCO2Wtr': func_list,
                #'pCH4Wtr': func_list,
                'cCO2Wtr': func_list,
                #'cCH4Wtr': func_list,
                #'dpCO2': func_list,
                #'dpCH4': func_list,
                #'pCO2Air': func_list,
                #'pCH4Air': func_list,
                'depth': ['mean'],
                #'TempWtr': func_list,
                param: func_list,}
res1 = res.groupby(['long', 'lat'], as_index=False).agg(cols_to_calc)
res1.columns = res1.columns.droplevel(1)
res1

In [ ]:


res1['square'] = mt.dist((0, res1['lat']), (0.02, res1['lat']))*1.11
res1

In [ ]:
grad=[-400, -200, -100, 0]
data = df_whole.loc[df_whole['depth']<0,:]
plt.rcParams['figure.figsize'] = [10, 10]
plt.plot('long', 'lat', data=obris, markersize=0.1, alpha=1)
cmap = plt.get_cmap('Blues_r', len(grad))
plt.scatter('long', 'lat', data=data, s=5, alpha=1, c=data['depth'], cmap=cmap, marker='s')
#plt.clim(-2000, 0)
plt.clim(np.min(grad)-0, np.max(grad)+0)
plt.colorbar(ticks=grad)
plt.title('depth, m')
#for yr in [2013, 2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024]:
#for yr in [2024]:
#    plt.scatter('long', 'lat', data=res.loc[res['year']==yr, :], s=20, alpha=0.1, c=res.loc[res['year']==yr, ('dpCO2', 'mean')], cmap='Paired')
    #plt.colorbar()
plt.savefig(f'DepthGrid7080.png')

In [ ]:
res1[f'{param}Level'] = np.nan
prev = 0
grad = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
for i in grad:
    print(f'{prev} -- {i}')
    res1.loc[res1[param].between(prev, i), f'{param}Level'] = i
    prev = i
    
# res1.loc[res1[param].between(-100, 100), f'{param}Level'] = 

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
plt.plot('long', 'lat', data=obris, markersize=1, alpha=1)
#plt.scatter('long', 'lat', data=df_whole, s=30, alpha=0.01, c=df_whole['depthLevel'], cmap='winter')
#plt.scatter('long', 'lat', data=df_whole.loc[df_whole['depth']<0], s=10, alpha=1, c='#eeeeff')
#plt.clim(-100, 500)
cmap = plt.get_cmap('rainbow', len(grad))
plt.scatter('long', 'lat', data=res, s=55, alpha=1, c=res[f'{param}'], cmap=cmap, marker='s')
plt.colorbar(ticks=grad, alpha=1)
plt.clim(np.min(grad)-0.25/2, np.max(grad)+0.25/2)
plt.title('CO2Wtr, мг/л')
#for yr in [2013, 2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024]:
#for yr in [2024]:
#    plt.scatter('long', 'lat', data=res.loc[res['year']==yr, :], s=20, alpha=0.1, c=res.loc[res['year']==yr, ('dpCO2', 'mean')], cmap='Paired')
    #plt.colorbar()
plt.grid(linewidth=1, alpha=0.5)
plt.savefig(f'{param}Grid7080Legend5_5km.png')

In [ ]:
func_list = ['count', 'mean', 'std', 'median']

cols_to_calc = {param: func_list,
                #'depth': func_list,
                #param: 'mean',
                'square': [sum],
               }
res2 = res1.groupby(['depthLevel'], as_index=False).agg(cols_to_calc)
#res2 = res2.groupby([('cCO2Wtr', 'count')], as_index=False).count()
res2.style.format(precision=3)


In [ ]:
res1['long']

In [ ]:
res1['square'] = mt.dist((0, res1['lat']), (0.02, res1['lat']))*1.11
res1.describe()

In [ ]:
res1['count'] = res1[('cCO2Wtr', 'count')]
res1.groupby(['count'], as_index=False).agg(['count'])


In [ ]:
res = res.groupby(['depthLevel'], as_index=False).agg(cols_to_calc)
res

In [ ]:
res.style.format(precision=2)

In [ ]:
destData

In [ ]:
res.to_csv(os.path.join(pwd, 'dpCO2CH4SeabySector.txt'),
           sep='\t',
           )

In [ ]:
df.to_csv(os.path.join(pwd, 'dpCO2CH4SeabyMinute.txt'),
           sep='\t'
          )

In [ ]:
%matplotlib inline
valMax = res[('cCH4Wtr','mean')].max

fig, ax = plt.subplots()
ax.set_xlim(106.2, 107.8)
ax.set_ylim(52, 53)

# wether this or those
ax.scatter(res['long'], res['lat'], c=res[('cCH4Wtr', 'mean', 'mean')], s=40, marker='s', linewidth=0, cmap='rainbow', alpha=0.1)
ax.plot(obris['long'], obris['lat'], 'b-')

fig.savefig('gis.png')

In [ ]:
df.info()

In [ ]:
df.to_csv(os.path.join(pwd, 'all.txt'), sep='\t')

In [ ]:
pwd

In [ ]:
import mytools

In [ ]:
mytools.dist((0,0), (1,0))

In [ ]:
import random
for i in range(256):
    a = random.randint(1, 10)
    b = random.randint(1, 10)
    c = a * b
    if random.randint(0, 3):
        print(f'{a} * {b} = ')
    else:
        print(f'{c} : {b} = ')


In [ ]:
d = pd.DataFrame({'year': [2021, 2022, 2023, 2024],
                  'a': [np.nan, 18.3, 21.4, 105],
                  'b': [np.nan, 31, 37, 41],
                  'c': [np.nan, 154.9, 257.3, 475.9],
                  'd': [np.nan, 447, 404, 518],
                  'e': [np.nan, 210, 193, 227],
                  'f': [np.nan, 142, 165, 173],
                  'h': [6, 10, 17, 20],
                  'i': [4.1, 18.3, 21.4, 105.1],
                 
                 
                 })
d

In [ ]:
col = 'i'
plt.rcParams['figure.figsize'] = [15, 8]
plt.plot('year', col, data=d, marker='o', markersize=30, linewidth=10)
plt.xticks(ticks=d['year'], minor=False)
plt.tick_params(axis='both', labelsize=40)
plt.xlim(2020.5, 2024.5)
plt.ylim(d[col].min()*0, d[col].max()*1.2)
plt.savefig(f'1.png')

In [ ]:
df.columns

In [ ]:
df = df.loc[:, ['DateTime', 'Longitude', 'Latitude', 'TempAir', 'PressAir', 'TempWtr',
                'pCO2Air', 'pCH4Air', 'pCO2Wtr', 'pCH4Wtr', 'depth', 'year', 
                'dateSec', 'dt', 
                'vesSpeed', 'winSpeed', 'dpCO2', 'dpCH4', 'region', 'Station', 'winSpeed_rolling']]